In [6]:
import os
import json
from collections import deque
from typing import Dict, List, Optional, Any, Union

from langchain_openai import ChatOpenAI


# Get human objective
# refine it via human in the loop and via adding context (short/long term memory search) -> the output is a json dict of the design
# build the modules and test benches usiiing short and long term memory

In [7]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [8]:
llm = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)
bigllm = ChatOpenAI(model='gpt-4-0125-preview', temperature=0)

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]

In [15]:
# modifying this such that the engine actually uses it
tools[0].description = 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions. Input should be a search query.'

# Context for FFT

In [28]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# FFT docs 
url = """https://en.wikipedia.org/wiki/Cooley%E2%80%93Tukey_FFT_algorithm"""
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Add 
#docs.extend([*docs_pydantic, *docs_sq])

# Sort the list based on the URLs in 'metadata' -> 'source'
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))

# Concatenate the 'page_content' of each sorted dictionary
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

# Requirement extraction chain

In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

requirement_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are very senior FPGA design engineer.
            Your goal is to expand the input objective into a requirement document for a design. You should also take into account the context provided.
            The design document consists of three items: "Goals", "Requirements", "HDL/HLS language".
            Keep the following in mind:
            1.  **Goal extraction**: Extracting the main goals based on the objective
            2.  **Technical Details**: Ensure that all technical information provided, including any specific features, instructions, or protocols, is thoroughly incorporated into the project's goals, requirements, and constraints.
            3.  **Focus on Planning**: Concentrate on defining the project's objectives and specifications. You are not responsible for writing any code, but rather for outlining the necessary steps and considerations for successful implementation.
            4.  **Focus on HDL/HLS design**: Come up with a step by step process for hardware description languages or high-level synthesis project. You are not responsible for anything other than logic design.
            5.  **Avoid generic specifications**: Avoid generic statements and ensure that your response comprehensively covers the technical aspects provided in the input or gathered through searches.
            6.  **Avoid non-functional specifications**: Do not include anything that is not related to HDL/HLS coding of the module. In other words, avoid time-lines, reliaility, scalability and such
            7.  **Self-Contained Response**: Ensure that your response is complete and standalone, suitable for input into another module without the need for external references. If you are refering to something you must include it in the output.
""",
        ),
        ("user", "Objective:\n {objective} \n\n Context: \n {context}"),
    ]
)

In [25]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.chains.openai_functions import create_structured_output_runnable

class Requirements(BaseModel):
    """Project requirements"""

    goals: List[str] = Field(
        description="List of goals based on the project's objectives"
    )

    requirements: List[str] = Field(
        description="List of requirements including all technical specifications and instructions provided"
    )

    lang: str = Field(
        description="HDL/HLS language to be used"
    )

requirement_runnable = create_structured_output_runnable(
    Requirements, bigllm, requirement_prompt
)

In [32]:
requirements = requirement_runnable.invoke(
    {"objective": "design a 32 bit 16-point FFT to run on an ultra-96 v2 FPGA board. the language must be system verilog.",
     "context" : concatenated_content}
)

type(requirements)

__main__.Requirements

In [35]:
print(requirements.requirements)

['The FFT module must support 32-bit wide data inputs and outputs.', 'The design must specifically target the Ultra-96 V2 FPGA board, taking into account its hardware specifications and constraints.', 'Implement the radix-2 DIT (decimation-in-time) version of the Cooley–Tukey FFT algorithm, as it is the simplest and most common form.', 'The design must handle 16 data points (N=16), assuming N is a power of two, which simplifies the implementation.', 'Optimize the design to achieve a computational complexity of O(N log N), where N is the number of data points.', 'Ensure that the design supports in-place computation to minimize memory usage.', 'Precompute twiddle factors to optimize performance.', 'Implement the design in SystemVerilog, utilizing its features for efficient hardware description.', 'The design should include a bit-reversal stage for data reordering, either as a pre-processing or post-processing step, depending on the chosen FFT implementation approach.', "Provide a testben

In [8]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4-turbo-preview")
# Construct the OpenAI Functions agent
agent_runnable = create_openai_tools_agent(llm, tools, prompt)

In [9]:
from langgraph.prebuilt import create_agent_executor

agent_executor = create_agent_executor(agent_runnable, tools)

agent_executor.invoke(
    {"input": "who is the winnner of the us open", "chat_history": []}
)

{'input': 'who is the winnner of the us open',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': "The winners of the US Open 2023 in different categories are as follows:\n\n- **Tennis (Men's Singles)**: Novak Djokovic won the men's singles title at the 2023 US Open by defeating Daniil Medvedev with a score of 6-3, 7-6(5), 6-3. This victory marked Djokovic's 24th Grand Slam singles title. [Source](https://www.usopen.org/en_US/news/articles/2023-09-10/novak_djokovic_wins_24th_grand_slam_singles_title_at_2023_us_open.html)\n\n- **Golf**: Wyndham Clark won the 2023 U.S. Open Golf Championship, edging out Rory McIlroy to claim his first major championship. This victory was also Clark's second PGA Tour victory. [Source](https://www.cbssports.com/golf/news/2023-u-s-open-leaderboard-wyndham-clark-breaks-through-edging-rory-mcilroy-for-first-major-championship/live/)\n\nPlease, let me know if you need information on a specific sport or further details."}, log="The winn

In [10]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [5]:
from openai import OpenAI
client = OpenAI()
import json
from tqdm import tqdm

l = {}
for T in tqdm(range(0, 11)):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {
        "role": "system",
        "content": "You will be provided with a high level idea of a hardware. Your overal task is to come up with an step by step approach for turing that idea into an actual synthesizable hardware. Currently you are expected to come up with 3 next steps in the idea exploration domain that you will return in a numbered list in markdown format. Do not talk about things that should be done such as literature review etc. Rather try to develop the idea."
        },
        {
        "role": "user",
        "content": "We want to build a competitor to GPU based on conventional logic. We want this new tool to beat GPUs for AI applications."
        }
    ],
    temperature=T*0.2,
    max_tokens=400,
    top_p=1
    )
    l[f'idea_{T}'] = {'T': T*0.2, 'content': response.choices[0].message.content}

out_file = open("test_gpt_3.5.json", "w") 
  
json.dump(l, out_file, indent = 6) 
  
out_file.close() 

100%|██████████| 11/11 [00:46<00:00,  4.21s/it]
